<a href="https://colab.research.google.com/github/AsraniSanjana/All_Codes/blob/main/All_Semester_Codes/bda_sem7_mp/movie_recommender_system-main/bda_mp_movielens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **BDA MINIPROJECT**

**GROUP NO.** : 10

**DIV**: D17B

**GROUP MEMBERS:**
1. SANJANA ASRANI, 01
2. NISHTHA BATRA, 07
3. VARAD DESHMUKH, 22
4. SAMARTH GAWALI, 25

#**TOPIC**: Trends in Recommendation & personalization in netflix


**dataset**: https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset?select=ratings.csv

#With Collaborative filtering we make predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption is that if a user A has the same opinion as a user B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a user chosen randomly.


In [ ]:
# !pip install pyspark

# creating a spark session

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('rec').getOrCreate()

# **Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using Alternating Least Squares.**

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

# using the movie lens rating dataset but a smaller version

In [ ]:
data = spark.read.csv('ratings_small.csv',inferSchema=True,header=True)

In [ ]:
data.head()

Row(userId=1, movieId=31, rating=2.5, timestamp=1260759144)

# describe.show to find min max avg of all cols

In [ ]:
data.describe().show()

+-------+------------------+------------------+------------------+--------------------+
|summary|            userId|           movieId|            rating|           timestamp|
+-------+------------------+------------------+------------------+--------------------+
|  count|            100004|            100004|            100004|              100004|
|   mean| 347.0113095476181|12548.664363425463| 3.543608255669773|1.1296390869392424E9|
| stddev|195.16383797819535|26369.198968815268|1.0580641091070326|1.9168582602710962E8|
|    min|                 1|                 1|               0.5|           789652009|
|    max|               671|            163949|               5.0|          1476640644|
+-------+------------------+------------------+------------------+--------------------+



# splitting the dataset

In [ ]:
# Smaller dataset so we will use 0.8 / 0.2
(training, test) = data.randomSplit([0.8, 0.2])

In [ ]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

In [ ]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

In [ ]:
predictions.show()

+------+-------+------+----------+----------+
|userId|movieId|rating| timestamp|prediction|
+------+-------+------+----------+----------+
|     1|     31|   2.5|1260759144| 1.8546026|
|     2|    296|   4.0| 835355395|  4.463022|
|     1|   2294|   2.0|1260759108| 1.5988243|
|     2|    253|   4.0| 835355511| 4.6869483|
|     2|    350|   4.0| 835355697| 2.5710425|
|     1|   1061|   3.0|1260759182| 1.4761487|
|     2|     47|   4.0| 835355552| 3.7448597|
|     1|   1172|   4.0|1260759205| 1.2133102|
|     2|     52|   3.0| 835356031| 3.4974098|
|     1|   2455|   2.5|1260759113| 2.0804927|
|     2|    168|   3.0| 835355710| 2.2973528|
|     2|    235|   3.0| 835355664|  4.640166|
|     1|   1029|   3.0|1260759179| 2.4428437|
|     2|    165|   3.0| 835355441| 3.4599445|
|     2|     10|   4.0| 835355493| 3.6740694|
|     2|     50|   4.0| 835355586| 3.8070698|
|     2|    480|   4.0| 835355643| 2.9965634|
|     2|    371|   3.0| 835355968| 3.0541942|
|     1|   1953|   4.0|1260759191|

In [ ]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = nan


# For a user with user-id 11, what will be the movie recommended?

In [ ]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [ ]:
# User had 10 ratings in the test data set
# Realistically this should be some sort of hold out set!
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     50|    11|
|   1027|    11|
|   1201|    11|
|  77455|    11|
|  91500|    11|
|  91548|    11|
+-------+------+



In [ ]:
reccomendations = model.transform(single_user)

In [ ]:
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|  91548|    11|       NaN|
|     50|    11|  3.970508|
|  77455|    11|  3.456606|
|   1201|    11| 2.9783525|
|  91500|    11| 2.5569963|
|   1027|    11| 1.8891362|
+-------+------+----------+

